# RA2CE feature: Single link losses analysis

## Single link losses Analysis

In [20]:
from pathlib import Path
import geopandas as gpd
from ra2ce.ra2ce_handler import Ra2ceHandler

# Import config data
root_dir = Path(r"C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses")
assert root_dir.exists()

# Load network data.
from ra2ce.network.network_config_data.network_config_data_reader import NetworkConfigDataReader
_network_config_data = NetworkConfigDataReader().read(root_dir.joinpath("network.ini"))
network_ini = root_dir / "network.ini"


# Load analysis data.
from ra2ce.analysis.analysis_config_data.analysis_config_data_reader import AnalysisConfigDataReader
_analysis_config_data = AnalysisConfigDataReader().read(root_dir.joinpath("analysis.ini"))

### Modify the analysis config data with custom properties

In [21]:
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses

# Modify the first analysis (assuming it's the only one)
_found_analysis: AnalysisSectionLosses = _analysis_config_data.analyses[0]
_input_analysis_path = root_dir.joinpath("input_analysis_data")
_found_analysis.traffic_intensities_file = _input_analysis_path.joinpath("traffic_intensities.csv")
_found_analysis.resilience_curve_file = _input_analysis_path.joinpath("resilience_curve.csv")
_found_analysis.values_of_time_file = _input_analysis_path.joinpath("values_of_time.csv")

### Run hazard overlay (configure)

In [22]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)

handler.configure()
# handler.run_analysis()

c:\Users\hauth\anaconda3\envs\ra2ce_env\lib\site-packages\osmnx\simplification.py:513: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = convert.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\hauth\anaconda3\envs\ra2ce_env\lib\site-packages\osmnx\simplification.py:560: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid
100%|██████████| 271/271 [00:00<00:00, 66292.80it/s]
2024-06-27 10:16:43 AM - [avg_speed_calculator.py:164] - root - WARNING - No valid file found with average speeds C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses\static\output_graph\avg_speed.csv, calculating and saving them instead.
2024-06-27 

In [23]:
hazard_output = root_dir / "static" / "output_graph" / "base_graph_hazard_edges.gpkg"
hazard_gdf = gpd.read_file(hazard_output, driver = "GPKG")
hazard_gdf.explore(column="EV1_ma", cmap = "viridis")

### Run the analysis

In [30]:
a = _analysis_config_data.network.file_id
print(a)
handler.run_analysis()

ID


KeyError: 'ID'

In [29]:
analysis_output_folder = root_dir / "output" / "single_link_losses" # specify path to output folder

losses_gdf = gpd.read_file(analysis_output_folder/"beira_single_losses.gpkg") #specify the name of the geopackage holding your results (can be found in the analysis output folder)
losses_gdf.head() #display the attributes of the file

2024-06-27 10:24:36 AM - [collection.py:162] - fiona._env - ERROR - C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses\output\single_link_losses\beira_single_losses.gpkg: No such file or directory
2024-06-27 10:24:36 AM - [collection.py:162] - fiona._env - ERROR - C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses\output\single_link_losses\beira_single_losses.gpkg: No such file or directory
2024-06-27 10:24:36 AM - [collection.py:162] - fiona._env - ERROR - C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses\output\single_link_losses\beira_single_losses.gpkg: No such file or directory
2024-06-27 10:24:36 AM - [collection.py:162] - fiona._env - ERROR - C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses\output\single_link_losses\beira_single_losses.gpkg: No such file or directory


DriverError: C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE\Beira Losses\output\single_link_losses\beira_single_losses.gpkg: No such file or directory

In [9]:
#Pick the right columns from the shapefile/geopackage
# losses['redundancy'] = losses['detour'].astype(str)
redundant_roads_map = losses_gdf.explore(column='vlh_EV1_ma_total', tiles="CartoDB positron",)
display(redundant_roads_map)